In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
import statsmodels.formula.api as smf
from statsmodels.tsa.holtwinters import SimpleExpSmoothing,ExponentialSmoothing, Holt
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.ar_model import AutoReg

# 实践中的统计
## 内华达州职业健康诊所  
在26个月里，它的月营业额从57000美元一直增长到超过3000000美元，直到主要门诊大楼被大火烧毁  
为了估计收入损失，诊所利用一种预测方法来推测在7个月的停业期间可能实现的营业增长。火灾前营业额的是数据将会成为预测模型的基础，该模型具有线性趋势和季节成分，这些成分将在本章中加以讨论  
假设我们要提供公司的一种产品在来年的季度销售预测。生产计划、原材料的采购、库存策略和销售定额都将受到我们提供的季度预测的影响  
预测方法可以分为定性和定量两种方法。定性方法通常包括利用专家判断来进行预测  
当以下条件同时满足时，可以使用定量预测方法：  
1. 被预测变量过去的信息可用
2. 这些信息可以被量化
3. 过去的模式将会持续到未来的假定合理

5  
如果历史数据局限于被预测变量的过去值，这种预测方法被称为时间序列方法，历史数据被称为时间序列。这种预测仅仅依赖于变量的过去值和过去的预测误差  
因果预测方法的依据是假定我们正在预测的变量与其他一个或几个变量存在一个因果关系。因此，如果我们能确定相关的自变量或解释变量的一个合适的集合，也许就能建立用于推测或预测时间序列估计的回归方程  
通过将时间视为自变量，时间序列视为应变量，回归分析也可以用于时间序列方法。为了区分回归分析在这两种情形下的应用，我们用术语截面回归和时间序列回归表述 5  
# 18.1 时间序列的模式
**时间序列** 是一个变量在连续时点或连续时期上策略的观测值的序列。数据的模式是了解时间序列过去行为的重要因素  
为了确定数据中的基本模式，有效的第一步是绘制**时间序列图**。时间序列图是时间和时间序列变量之间关系的图形表述：时间位于横轴，时间序列值位于纵轴  
## 18.1.1 水平模式
当数据围绕着一个不变的均值上下波动时，则存在**水平模式**。为了说明时间序列具有水平模式，考虑表18-1中12周的数据。图18-1是这些数据的时间序列图  

In [ ]:
'''
5
python	pandas	series	pd.Series() index
python	pandas	series	pd.Series() name

'''
gas_sale=pd.Series(data=[17,21,19,23,18,16,20,18,22,20,15,22],
                   index=pd.Index(np.arange(1,13),name='week'),
                   name='sales')
gas_sale.head()

In [ ]:
gas_sale.plot()
plt.scatter(gas_sale.index,gas_sale)
plt.xticks(np.arange(0,13))
plt.ylim(0,25)
plt.xlim(0,12.1)
plt.show()

术语**平稳时间序列**是指统计性质与时间独立的时间序列。特别地，其含义为  
1. 过程产生的数据有一个不变的均值  
2. 时间序列的变异性随着时间的推移保持不变 5  

平稳时间序列的时间序列图总是显示出水平模式，但仅仅观测到水平模式就得出时间序列是平稳的结论，并没有足够的证据  
经营环境的改变常常导致具有水平模式的时间序列移动到一个新的水平。例如，假设汽油经销商与佛蒙特州警方签署了一份合同，给佛蒙特州南部的警车加油。由于这个新合同，经销商希望看到从第13周开始周销售量有大幅度的增加  
表18-2给出了汽油销售量的原始时间序列以及签署了新合同后10周的销售量数据。图18-2是想要的时间序列图    

In [ ]:
'''
5
python	pandas	series	s.tail()
'''
gas_sale_lv2=pd.Series(data=[17,21,19,23,18,16,20,18,22,20,15,22,31,34,31,33,28,32,30,29,34,33],
                   index=pd.Index(np.arange(1,23),name='week'),
                   name='sales')
gas_sale_lv2.tail()

In [ ]:
gas_sale_lv2.plot(figsize=(10,7))
plt.scatter(gas_sale_lv2.index,gas_sale_lv2)
plt.xticks(np.arange(0,25))
plt.ylim(0,40)
plt.xlim(0,24)
plt.show()

## 18.1.2 趋势模式
尽管时间序列的数据通常呈现随机起伏的状态，但在一段较长的时间内，它仍然呈现出逐步的改变或移动到相对较高或较低的值。如果时间序列图显示出这里形态特征，我们则称存在**趋势模式**  
趋势通常是长期因素影响的结果，例如人口总数的增加或减少、人口总体特征、工艺和顾客偏爱的变化等  
为了说明具有趋势模式的时间序列，我们考虑某制造商过去10年自行车销售量的时间序列，如表18-3和图18-3所示 5  

In [ ]:
bic=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch17/Bicycle.csv')
bic_sale=bic.set_index('Year')['Sales']
bic_sale.head()

In [ ]:
bic_sale.plot()
plt.scatter(bic_sale.index,bic_sale)
plt.xticks(np.arange(0,13))
plt.ylim(20,34)
plt.xlim(0,12)
plt.show()

自行车销售时间序列的趋势呈现线性且随着时间推移而增长，但有时趋势能用其他类型的模式更好地描述 5  
例如，表18-4是从10年前公司的胆固醇药物获得FDA批准后的销售收入数据，图18-4是相应的时间序列图。时间序列以非线性的方式增长，即每年收入的变化率不是以一个固定的数值来增加  
事实上，当收入呈现指数形式的增长，环比变化率从一个时期到下一个时期相对稳定时，这样的指数关系是合适的  

In [ ]:
cho=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch17/Cholesterol.csv')
cho_sale=cho.set_index('Year')['Revenue']
cho_sale.head()

In [ ]:
'''
5
'''
cho_sale.plot()
plt.scatter(cho_sale.index,cho_sale)
plt.xticks(np.arange(0,11))
plt.ylim(0,120)
plt.xlim(0,10.1)
plt.show()

## 18.1.3 季节模式
时间序列的趋势是通过分析历史数据多年的移动来识别的。毫无疑问，**季节模式**是指，在超过一年的时期内，由于受季节的影响，时间序列图呈现重复模式  
尽管我们通常认为时间序列的季节变动是在一年内出现，但在小于一年的时期内时间序列数据也可能呈现季节模式  
表18-5是时间序列，图18-5是相应的时间序列图。时间序列图没有显示销售量有长期趋势  

In [ ]:
umb=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch17/Umbrella.csv')
umb.head(10)

In [ ]:
'''
5
python	pandas	dataframe	d.fillna() method
python	pandas	dataframe	d.astype()
'''
umb_lv1=umb.fillna(method='ffill').astype(int)
umb_lv1['year_Q']=umb_lv1['Year'].astype(str) + 'Q' +umb_lv1['Quarter'].astype(str) 
umb_lv1.head()

In [ ]:
umb_lv1.plot('year_Q','Sales',figsize=(10,7))
plt.scatter(umb_lv1['year_Q'],umb_lv1['Sales'])
plt.ylim(0,180)
plt.xlim(0,len(umb_lv1['year_Q']))
plt.xticks(np.arange(0,len(umb_lv1['year_Q'])+1),umb_lv1['year_Q'])
plt.show()

因此，我们得出结论：存在季度的季节模式 5  
## 18.1.4 趋势与季节模式
有时时间序列同时包含趋势模式和季节模式。例如，表18-6的数据是某制造商过去4年的电视机销售量，图18-6是相应的时间序列图  

In [ ]:
'''
python	pandas	dataframe	d.tail()
'''
tv_sale=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch17/TVSales.csv')
tv_sale.columns=['Year','Quarter','Sales']
tv_sale.tail()

In [ ]:
tv_sale['year_Q']=tv_sale['Year'].astype(str) + 'Q' +tv_sale['Quarter'].astype(str) 

tv_sale.head()


In [ ]:
'''
5
'''
tv_sale.plot('year_Q','Sales',figsize=(10,7))
plt.scatter(tv_sale['year_Q'],tv_sale['Sales'])
plt.ylim(0,9)
plt.xlim(0,len(tv_sale['year_Q']))
plt.xticks(np.arange(0,len(tv_sale['year_Q'])+1),tv_sale['year_Q'])
plt.show()

## 18.1.5 循环模式
如果时间序列图显示出持续时间超过一年的趋势线的上下交替的点序列，则存在**循环模式**  
许多经济事件序列存在循环行为，其观测值围绕着趋势线有规则地上下波动。通常，时间序列的循环成分归因于多年的经济周期  
经济周期的预测，如果不是不可能，也是极为困难的。因此，循环影响常常与长期趋势影响合并，称为趋势循环影响 5  
## 18.1.6 选择预测方法  
时间序列的基本模式是选择预测方法的重要因素。因此，当试图确定使用怎样的预测方法时，第一项工作应该是绘制时间序列图 5  
# 18.2 预测精度
本节我们首先对表18-1汽油销售量的时间序列用所有预测方法中最简单的一种方法进行预测，即用最近一周的销售量作为下一周的预测值  
用这种方法得到的预测值列在表18-7预测值一列中。由于其简单，故该方法常常被称为朴素预测法  

In [ ]:
gas_sales=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch17/Gasoline.csv')
gas_sales.head()

In [ ]:
gas_sales_lv1=gas_sales.set_index('Week')
gas_sales_lv1.head()

In [ ]:
'''
5
'''
gas_sales_lv2=gas_sales.set_index(gas_sales['Week']+1)['Sales']

gas_sales_lv2.name='Estimate'
gas_sales_lv2.head()

In [ ]:
'''
python	pandas	series	s.join()

'''
gas_sales_df=gas_sales_lv1.join(gas_sales_lv2)
gas_sales_df.head()

In [ ]:
'''

python	pandas	series	s.sub()

'''
gas_sales_df['error']=gas_sales_df['Sales'].sub(gas_sales_df['Estimate'])
gas_sales_df.head()

In [ ]:
'''
python	pandas	series	s.abs()
'''
gas_sales_df['error_abs']=gas_sales_df['error'].abs()
gas_sales_df.head()

In [ ]:
gas_sales_df['error_sqr']=gas_sales_df['error'].pow(2)
gas_sales_df.head()

In [ ]:
'''
5
'''
gas_sales_df['error_perc']=gas_sales_df['error'].div(gas_sales_df['Sales']).mul(100).round(2)
gas_sales_df.head()

In [ ]:
gas_sales_df['error_perc_abs']=gas_sales_df['error_perc'].abs()
gas_sales_df.head()

对于已知数据，通过选择最佳精度的方法，我们希望增加获得未来时期更好预测的可能性  
与测量预测精度相关的重要概念是**预测误差**，其定义为  
<center>预测值=实际值-预测值</center>

例如，由于经销商第2周实际售出21千加仑汽油，而用第1周销售量得到的预测值是17千加仑，因此第2周的预测误差为 5  

In [ ]:
print('第2周的预测误差={:.0f}'.format(gas_sales_df.loc[2,'error']))

因此，误差可正可负，取决于预测值是太低还是太高。表18-7中预测误差一列是朴素预测法预测误差的完整汇总  
一个简单的预测精度的测度是预测误差的均值或平均数。由表18-7可知，

In [ ]:
'''
5
'''
error_sum=gas_sales_df['error'].sum()
error_mean=gas_sales_df['error'].mean()
print('汽油销售量时间序列预测误差之和为{:.0f},因此，预测误差的均值为{:.2f}'.format(error_sum,error_mean))

由于正的和负的预测误差相互抵消，平均误差应该很小，因此，平均误差不是预测精度的常用测度  
**平均绝对误差**是避免正负预测误差相互抵消的一种预测精度的测度，记为MAE。正如其名，MAE是预测误差绝对值的平均数  
由表18-7可知，预测误差绝对值之和为41，因此  

In [ ]:
print('MAE=预测误差绝对值的平均数={:.2f}'.format(gas_sales_df['error_abs'].mean()))

避免正负预测误差相互抵消的另一种测量方法是计算预测误差平方的平均数。这种预测精度的测度称为**均方误差**，记为MSE 5    

In [ ]:
error_sqrsum=gas_sales_df['error_sqr'].sum()
error_sqrmean=gas_sales_df['error_sqr'].mean()
print('由表18-7可知，误差平方和为{:.0f}，因此MSE=预测误差平方和的平均数={:.2f}'.format(error_sqrsum,
                                                             error_sqrmean))

MAE和MSE的大小依赖于数据的测量尺度。因此，对不同时间间隔进行比较是困难的，例如对预测月汽油销售量的方法和预测周销售量的方法进行比较，或对不同的时间序列进行比较都是困难的  
为了对这些情况进行比较，我们需要考虑相对的或百分数误差的测度。**平均绝对百分数误差**记为MAPE，就是这样一种测度    

In [ ]:
print('第2周的百分数误差={:.2f}%'.format(gas_sales_df.loc[2,'error_perc']))

百分数误差的完整汇总列在表18-7百分数误差一列中。我们还给出了百分数误差的绝对值 5  

In [ ]:
error_perc_abssum=gas_sales_df['error_perc_abs'].sum()
error_perc_absmean=gas_sales_df['error_perc_abs'].mean()
print('由表18-7可知，百分数误差的绝对值之和为{:.2f},因此MAPE=百分数预测误差的绝对值的平均数={:.2f}%'\
      .format(error_perc_abssum,error_perc_absmean))

总之，用朴素预测法，我们得到如下预测精度的测度：  
MAE=3.73  
MSE=16.27  
MAPE=19.24%  
这些预测精度的测度简单地度量了预测方法任何能很好地预测时间序列的历史数值  
但是，如果我们选择一种适用于历史数据的预测方法，而且我们相信历史模式可以持续到未来，那么我们应该得到最终将被证明是好的结果 5  
在结束本节之前，我们考虑表18-1汽油销售量时间序列的另一种预测方法。假设我们用所有可得到的历史数据的平均值作为下一期的预测值    
用这种方法得到的汽油时间序列的预测值在表18-8的预测值一列中 

In [ ]:
gas_sales_copy1=gas_sales_lv1.join(gas_sales_lv2).copy()
gas_sales_copy1.columns=['Sales','simple_estimate']
gas_sales_copy1.head()

In [ ]:
gas_sales_copy1['num']=np.arange(0,len(gas_sales_copy1))
gas_sales_copy1.head()

In [ ]:
'''
5
python	pandas	series	s.cumsum()
'''
gas_sales_copy1['cumsum']=gas_sales_copy1['simple_estimate'].cumsum()
gas_sales_copy1.head()

In [ ]:
gas_sales_copy1['cummean']=gas_sales_copy1['cumsum'].div(gas_sales_copy1['num']).round(2)
gas_sales_copy1.head()

In [ ]:
gas=gas_sales_copy1.loc[:,['Sales','cummean']].copy()
gas.columns=['Sales','Estimate']
gas.head()

In [ ]:
def simplePredictionMethod(X,y):
    '''
    统计常用代码	时间序列分析及预测	朴素预测法	朴素预测法
    '''
    df=pd.DataFrame([X,y]).T
    df['error']=X.sub(y)
    df['error_abs']=df['error'].abs()
    df['error_sqr']=df['error'].pow(2).round(2)
    df['error_perc']=df['error'].div(X).mul(100).round(2)
    df['error_perc_abs']=df['error_perc'].abs()
    return df

gas_df=simplePredictionMethod(gas['Sales'],gas['Estimate'])
gas_df

In [ ]:
gas_df.sum()

In [ ]:
'''
5  
'''
gas_df_mean=gas_df.mean().round(2)
gas_df_mean

现在我们通过比较每一种方法的MAE、MSE和MAPE的值，来比较本节介绍的两种预测方法的精度  
对每一种测度，过去数值平均数法都比用最近一期观测值作为下一期预测值的方法提供了更准确的预测。一般地，如果时间序列基本上是平稳的，所有历史数据平均值法将永远提供最好的结果    

In [ ]:
'''
python	pandas	Index	pd.Index() name
5
'''
gas_mae=gas_df_mean['error_abs']
gas_mse=gas_df_mean['error_sqr']
gas_mape=gas_df_mean['error_perc_abs']
gas_sample_mae=round(gas_sales_df['error_abs'].mean(),2)
gas_sample_mse=round(error_sqrmean,2)
gas_sample_mape=round(error_perc_absmean,2)
gas_prd=pd.DataFrame({'朴素预测法':[gas_sample_mae,gas_sample_mse,gas_sample_mape],
             '过去数值平均数法':[gas_mae,gas_mse,gas_mape]},index=pd.Index(['MAE','MSE','MAPE'],name='评估指标'))
gas_prd

假设基本的时间序列是不平稳的  
当这种移动到新水平的情况出现时，用所有历史数据的平均值法调整到时间序列的新水平需要用很长时间。在这种青霞，简单的朴素方法对水平的改变调整迅速，因为它用最近一期的观测值作为预测值  
在比较不同的预测方法时，预测精度的测度是一个重要因素。同时历史预测精度不是唯一的考虑因素，时间序列在未来有可能改变的情形尤其如此  
预测方法快速适应水平改变的能力是重要的考虑因素，尤其是对于短期预测情形 5  
# 18.3 移动平均法和指数平滑法  
本节我们将讨论适合于有水平模式的时间序列的3种预测方法：移动平均法、加权移动平均法和指数平滑法  
这些方法能很好地适应水平模式的时间序列水平的改变，比如我们看到的汽油销售量的扩展时间序列。但是，当有明显的趋势、循环或季节影响时，这些方法未经过修正是不适用的  
由于每一种方法的目的是为了“消除”时机序列的随机波动，因此它们都被称为平滑方法。这些方法易于使用，通常对短期预测，例如下一个时期的预测，可以提供较高的精度水平  
## 18.3.1 移动平均法
**移动平均法**使用时间序列中最近k期数据值的平均数作为下一个时期的预测值。在数学上，k阶移动平均预测如下：  
<hr />

**k阶移动平均预测**（18-1） 5  
$$F_{t+1}=\frac{Y_t+Y_{t-1}+\dots+Y_{t-k+1}}{k}$$
式中，$F_{t+1}$是时间序列t+1期的预测值;$Y_t$是时间序列t期的实际值  
<hr />

术语移动的含义：每次使用时间序列一个新的观测值，用它代替公式中最旧的观测值，并且计算出一个新的平均数  
图18-1的时间序列图表明汽油销售量时间序列具有水平模式。因此，本节的平滑方法是适用的  
于是，较小的k值将更快速追踪时间序列的移动，而较大的k值将随着时间的推移更有效地消除随机波动。因此，以对时间序列台式的了解为依据的管理判断，是有助于选择一个合适的k值 5  
为了说明如何用移动平均法预测汽油的销售量，我们将使用3周移动平均(k=3)  
$$F_4=第1-3周的平均数=(17+21+19)/3=19$$
接下来，我们用第2-4周时间序列的平均数计算第5周销售量的预测值  
表18-9给出了汽油销售量时间序列的3周移动平均预测值的全部汇总，图18-7是原始时间序列图和3周移动平均预测图  

In [ ]:
'''
5
python	pandas	series	s.rolling()
python	pandas	Window	Rolling.mean()

'''
gas_rolling_mean=gas_sales_lv1.rolling(3).mean()
gas_rolling_mean.columns=['Estimate']

gas_rolling=gas_sales_lv1.join(gas_rolling_mean.set_index(gas_sales['Week']+1))
gas_rolling.head()

In [ ]:
'''
统计常用代码	时间序列分析及预测	预测精度	预测数据
'''
gas_rol_df=simplePredictionMethod(gas_rolling['Sales'],gas_rolling['Estimate'])
gas_rol_df

In [ ]:
gas_rol_df['Estimate']

In [ ]:
'''
python	pandas	Index	i.values
'''
gas_rol_df.plot(y=['Sales','Estimate'])
plt.scatter(x=gas_rol_df.index.values,y=gas_rol_df['Sales'])
plt.scatter(x=gas_rol_df.index.values,y=gas_rol_df['Estimate'])
plt.ylim(0,25)
plt.xlim(0,12)
plt.show()

**预测精度**  在18.2节我们讨论了3种预测精度的测度：MAE、MSE和MAPE。利用表18-9的3周移动平均的计算结果，3种预测精度的测度分别是 5  

In [ ]:
def predAcc(dx,dy):
    '''
    统计常用代码	时间序列分析及预测	预测精度	预测精度
    '''
    df=simplePredictionMethod(dx,dy)
    mae=round(df['error_abs'].mean(),2)
    mse=round(df['error_sqr'].mean(),2)
    mape=round(df['error_perc_abs'].mean(),2)
    return pd.DataFrame({'accuracy':[mae,mse,mape]},
                        index=pd.Index(['MAE','MSE','MAPE'],name='method'))

predAcc(gas_rolling['Sales'],gas_rolling['Estimate'])

在18.2节我们还给出了用最近的观测值作为下一周预测值k=1阶的移动平均的结果

In [ ]:
gas_prd

因此，在每一种情形下，3周移动平均方法都比简单地用最近的观测值作为预测值给出了更准确的预测  
为了确定使用不同的阶数k的移动平均数是否可以给出更准确的预测，我们建议用试验和误差来确定使MSE达到最小的k值 5  
## 18.3.2 加权移动平均法
在移动平均法中，移动平均数计算中的每个观测值都得到相同的权重  
一种被称为**加权移动平均**的方法对此做了改变，即对每个数据值选择不同的权重，然后计算最近k期数据值得加权平均数作为预测值  
在大多数情况下，最近的观测值得到最大得权重，而减少较远期观测值的权重  
注意，加权移动平均方法的权重之和应该等于1  
**预测精度** 为了使用加权移动平均法，我们首先应该选择加权移动平均数中所包含的数据值个数，然后对每个数据值选择权重 5  
选择权重的唯一要求是它们的总和必须等于1。为了确定数据值个数和权重的一个特定组合是否比其他组合提供了更精确的预测，我们建议使用MSE作为预测精度的测度  

In [ ]:
'''
python	pandas	series	s.rolling() win_type
'''
gas_sales_lv1.rolling(3,win_type='boxcar').mean().head()

In [ ]:
'''
python	scipy	Signal processing (scipy.signal)	get_window()
python	scipy	Signal processing (scipy.signal)	get_window() boxcar
'''
signal.get_window('boxcar', 3)

In [ ]:
'''
python	scipy	Signal processing (scipy.signal)	get_window() blackman
'''
signal.get_window('blackman', 15).round(2)

## 18.3.3 指数平滑法
**指数平滑法** 也是利用过去的时间序列值得加权平均作为预测值;它是加权移动平均法得一个特例，即我们只选择一个权重——最近时期观测值的权重 5  
其他数据值得权重可以自动推算出来，并且随着观测值距离预测期越远，权数也变得越小。指数平滑法方程如下  
<hr />

**指数平滑预测**（18-2）  
$$F_{i+1}=\alpha Y_t+(1-\alpha)F_i$$
<hr />

式中，$F_{i+1}$是时间序列t+1期的预测值;$Y_i$是时间序列t期的实际值。$F_t$是时间序列t期的预测值;$\alpha$是平滑常数($0\le\alpha\le1$)  
<hr />

式（18-2）表明t+1期的预测值是t期的实际值和预测值的加权平均数，t期实际值的权重为**平滑常数**$\alpha$,t期预测值的权重为$1-\alpha$ 5  
可以证明，任何时期的指数平滑预测值实际上是时间序列所有过去实际数据的一个加权平均数  
开始计算，我们令$F_1$等于时间序列1期的实际值，即$F_1=Y_1$。因此，2期的预测值为  
$$F_2=\alpha Y_1+(1-\alpha)F_1=Y_1$$  
我们看到，2期的指数平滑预测值等于时间序列1期的实际值  
3期的预测值为$F_3=\alpha Y_2+(1-\alpha)Y_1$ 5  
最后，将$F_3$的表达式代入$F_4$的表达式中，我们得到  
$$F_4=\alpha Y_3+\alpha(1-\alpha)Y_2+(1-\alpha)^2Y_1$$
现在我们看到，$F_4$是前三个时间序列数值的加权平均数。$Y_1,Y_2和Y_3$的系数或权重之和等于1  
在一般情形下，一个类似的结论可以表述为：任何预测值$F_{t+1}$是所有时间序列过去数值的加权平均数  
事实上，式(18-2)表明，一旦选定了平滑常数$\alpha$的值，我们仅仅需要两项信息就可计算预测值：t期时间序列的实际值$Y_t$和t期的预测值$F_t$ 5  
为了说明指数平滑法，让我们再次考虑表18-1和图18-1中的汽油销售量时间序列  
图18-8是时间序列的实际值和预测值的图。尤其要注意的是，预测值“消除”了时间序列中的不规则或随机波动  

In [ ]:
'''
python	pandas	series	s.ewm() adjust
python	pandas	Window	ExponentialMovingWindow.mean()
'''
gas_ewm=gas_sale.ewm(alpha=0.2,adjust=False).mean().round(2)
gas_ewm.name='Estimate'
gas_ewm.head()

In [ ]:
gas_ewm_df=simplePredictionMethod(gas_sale,gas_ewm)
gas_ewm_df

In [ ]:
'''
5
'''
gas_ewm_df.plot(y=['sales','Estimate'])
plt.scatter(x=gas_ewm_df.index.values,y=gas_ewm_df['sales'])
plt.scatter(x=gas_ewm_df.index.values,y=gas_ewm_df['Estimate'])
plt.ylim(0,25)
plt.xlim(0,12)
plt.show()

**预测精度** 在前面的指数平滑计算中，我们使用的平滑常数$\alpha=0.2$。尽管$\alpha$取0和1之间的任何值都是可以接受的，但是有些$\alpha$值得到的预测值比其他$\alpha$值更好  
为了贯彻如何得到一个合适的$\alpha$值,将基本指数平滑模型改写如下：（18-3） 
$$F_{t+1}=\alpha Y_t+(1-\alpha)F_t$$
$$F_{t+1}=\alpha Y_t+F_t-\alpha F_t$$
$$F_{t+1}=F_t+\alpha(Y_t-F_t)$$
因此，新的预测值$F_{t+1}$等于过去的预测值$F_t$加上一个调整值，这个调整值等于$\alpha$乘以最近时期的预测误差$(Y_t-F_t)$。即通过调整t期的预测值和一部分预测误差，我就可以得到t+1期的预测值 5  
如果时间序列包含大包含大量的随机波动，我们倾向于选择较小的平滑常数。这样选择的原因是：大多数预测误差是由随机波动引起的，我们不希望对预测做出过的反应和过快的调整  
对随机波动相对较小的时间序列，预测误差更可能表示序列水平的改变。于是，选择较大的平滑常数的优点是迅速调整预测，这使得预测对改变的条件反应的得更迅速  
我们用来确定平滑常数$\alpha$合理值的准则，与我们建议的确定包含在移动平均计算中的阶数或数据时期个数的准则相同  
就较小的MSE值而言，不同的$\alpha$能否提供一个较好的结果呢？也许回答这个问题最简单的方法是用另一个$\alpha$值来试验，然后将所得到的MSE与用平滑常数$\alpha=0.2$得到的MSE=8.98进行比较  
表18-11是用$\alpha=0.3$得到的指数平滑预测结果 5  

In [ ]:
'''
统计常用代码	时间序列分析及预测	预测精度	预测精度
'''
predAcc(gas_sale,gas_ewm)

In [ ]:
'''
python	pandas	series	s.ewm() adjust
python	pandas	Window	ExponentialMovingWindow.mean()
'''
gas_ewm=gas_sale.ewm(alpha=0.3,adjust=False).mean().round(2)
gas_ewm.name='Estimate'
gas_ewm.head()

In [ ]:
gas_ewm_df=simplePredictionMethod(gas_sale,gas_ewm)
gas_ewm_df

In [ ]:
predAcc(gas_sale,gas_ewm)

In [ ]:
'''
python	statsmodels	Time Series analysis tsa	SimpleExpSmoothing()
python	statsmodels	Time Series analysis tsa	SimpleExpSmoothing.fit()
python	matplotlib	lines	marker
python	statsmodels	Time Series analysis tsa	HoltWintersResults.forecast()
python	statsmodels	Time Series analysis tsa	HoltWintersResults.fittedvalues
python	matplotlib	Pyplot function overview	plt.legend() handles
python	matplotlib	Pyplot function overview	plt.legend() labels
'''
fit3=SimpleExpSmoothing(gas_sale.values).fit()
l3, = plt.plot(list(fit3.fittedvalues) + list(fit3.forecast(5)), marker='o')
l4, = plt.plot(gas_sale, marker='o')
plt.ylim(0,25)
plt.xlim(0,12)
plt.legend(handles = [l3, l4], labels = ['auto', 'gas_sale'])
plt.show()

以后的数据，即我们得到时间序列新的观测值以后，我们分析新搜集的时间序列数据，决定是否应调整平滑常数以得到更好地预测结果 5  
## 注释
1. 电子制表软件包是一种有效的辅助工具，可以用于选择指数平滑法中合适的$\alpha$值  
2. 我们在平稳时间序列的背景下讨论移动平均法和指数平滑法。这些方法也可以用于水平改变但没有呈现趋势或季节的非平稳时间序列的预测 5  

# 18.4 趋势推测法
本节我们介绍三种适合于具有趋势模式的时间序列的预测方法  
## 18.4.1 线性趋势回归
在18.1节我们用表18-3和图18-3的自行车销售量的时间序列，来说明具有趋势模式的时间序列。自行车销售量时间序列的数据再一次出现在表18-12和图18-9中  

In [ ]:
bicyle=bic.set_index('Year')
print(len(bicyle))
bicyle.head()

In [ ]:
'''
python	patsy	formula	~
python	statsmodels	statsmodels.formula.api	ols()
python	statsmodels	statsmodels.formula.api	Models.fit()

'''
bic_formula_lv1='Sales ~ Year'
bic_mod=smf.ols(bic_formula_lv1,bic).fit()
bic_mod.params

In [ ]:
'''
5
'''
bic_int=bic_mod.params[0]
bic_cef=bic_mod.params[1]
bicyle.plot()
plt.scatter(bicyle.index.values,bicyle)
plt.xlim(0,12)
plt.ylim(20,34)
plt.plot(np.array([0,12]),np.array([bic_int,bic_int+bic_cef*12]))
plt.xticks(np.arange(0,13,1))
plt.show()

尽管图18-9的时间序列图显示在过去10年有一些上下波动，但我们可能会同意，图18-10所示的线性趋势线对时间序列的长期变化提供了一个合理的近似   
在第14章，描述自变量x和应变量y之间直接关系的估计回归方程为  
$$\hat{y}=b_0+b_1x$$
因此，对时间序列的线性趋势，我们将利用下面的估计的回归方程  
<hr />

**线性趋势方程**(18-4)  
$$T_t=b_0+b_1t$$
5  
式中，$T_t$是t期线性趋势预测值;$b_0$是线性趋势线的截距;$b_1$是线性趋势线的斜率;t是期间  
<hr />

式（18-4）中的估计的回归系数($b_0和b_1$)的计算公式如下  
<hr />

**计算线性趋势的斜率和截距的公式**（18-5）  
$$b_1=\frac{\sum_{t=1}^n{(t-\bar{t})(Y_t-\bar{Y})}}{\sum_{t=1}^n{(t-\bar{t})^2}}$$
(18-6)  
$$b_0=\bar{Y}-b_1\bar{t}$$
5  
式中，$Y_t$是t期时间序列的值;n是时期个数（观测值的个数);$\bar{Y}$是时间序列的平均值;$\bar{t}$是t的平均值  
<hr />

为了计算趋势推测方法的精度，我们将使用均方误差MSE  

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.fittedvalues
'''
bic_Estimate=bic_mod.fittedvalues
bic_Estimate.name='Estimate'
bic_Estimate.index=bicyle.index
len(bic_Estimate)
bic_Estimate.head()

In [ ]:
predAcc(bicyle['Sales'],bic_Estimate)

In [ ]:
'''
5
python	statsmodels	Linear Regression	RegressionResults.mse_resid
'''
print('在图18-11的方差分析（ANOVA）表中，MSE的值为{:.2f}'.format(bic_mod.mse_resid))

这个MSE值与我们前面计算的MSE不同，原因在于此误差平方和除以8而不是10;于是，在回归输出中MSE不是预测误差平方的平均数  
## 18.4.2 Holt线性指数平滑
查尔斯·霍尔特建立了用于预测具有线性趋势的时间序列的指数平滑形式。回顾18.3节讨论的指数平滑方法，它 使用平滑常数$\alpha$“消除”时间序列中随机的或不规则影响;并且用方程  
$$F_{t+1}=\alpha Y_t+(1-\alpha)F_t$$
得到t+1期的预测值  
**Holt线性指数平滑**方法用两个平滑常数$\alpha,\beta$和三个方程得到预测  
<hr />

**Holt线性指数平滑方程** 5  
(18-7)  
$$L_t=\alpha Y_t+(1-\alpha)(L_{t-1}+b_{t-1})$$
(18-8)
$$b_t=\beta(L_t-L_{t-1})+(1-\beta)b_{t-1}$$
(18-9)
$$F_{t+k}=L_t+b_1k$$
式中，$L_t$是t期时间序列水平的估计值;$b_t$是t期时间序列斜率的估计值;$\alpha$是时间序列水平的平滑常数;$\beta$是时间序列斜率的平滑常数;$F_{t+k}$是向前k期的预测值；k是向前预测的时期个数  
<hr />

让我们对表18-12的自行车销售量时间序列，应用$\alpha=0.1,\beta=0.2$的Holt方法。为了启动该方法，我们需要时间序列第1年的水平估计值$L_1$和时间序列第1年的斜率估计值$b_1$ 5  

In [ ]:
'''
python	statsmodels	Time Series analysis tsa	Holt() 
python	statsmodels	Time Series analysis tsa	Holt.fit() smoothing_level/smoothing_slope
python	numpy	The N-dimensional array (ndarray)	ndarray.round()

'''
bic_v=bicyle['Sales'].values
bic_mod=Holt(bic_v,damped=False).fit(smoothing_level=0.2)
bic_fit_v=list(bic_mod.fittedvalues.round(2))
bic_fit_v

用类似的方式进行计算，结果显示在表18-15中

In [ ]:
'''
python	pandas	series	pd.Series() index
python	pandas	series	pd.Series() name
'''
bic_estimate_lv2=pd.Series(bic_fit_v,name='Estimate',index=bicyle.index)
bic_estimate_lv2

In [ ]:
'''
统计常用代码	时间序列分析及预测	预测精度	预测数据
'''
bic_df=simplePredictionMethod(bicyle['Sales'],bic_estimate_lv2)
bic_df

In [ ]:
predAcc(bicyle['Sales'],bic_estimate_lv2)

In [ ]:
'''
5
'''
bic_mod.forecast(1)

## 18.4.3 非线性趋势回归
用线性函数来拟合趋势是很常见的。然而，正如我们前面讨论的，有时时间序列有趋向或非线性趋势。表18-16是时间序列，图18-13是相应的时间序列图  

In [ ]:
chol=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch17/Cholesterol.csv')
chol

In [ ]:
chol.plot(x='Year',y='Revenue',marker='o')
plt.xlim(0,10)
plt.ylim(0,120)
plt.xticks(np.arange(0,11))
plt.show()

但是与自行车销售量时间序列不同，线性趋势似乎并不合适。相反，在拟合长期趋势时，似乎需要一个曲线函数  
**二次趋势方程**  各种非线性函数能用来建立胆固醇时间序列的趋势估计。例如，考虑下面的二次趋势方程（18-10）： 5  
$$T_t=b_0+b_1t+b_2t^2$$  
图18-14显示了二次趋势模型的Minitab多元回归输出结果;估计的回归方程是  

In [ ]:
chol_formula='Revenue ~ Year + np.power(Year,2)'
chol_mod=smf.ols(chol_formula,chol).fit()
chol_mod.params
print('销售收入100万美元={:.1f}{:+.2f}Year{:+.3f}YearSq'.format(chol_mod.params[0],
                                                  chol_mod.params[1],
                                                  chol_mod.params[2]))

当解决涉及二次趋势的练习题时，我们推荐使用Trend Analysis这种方法  
**指数趋势方程** 另一种用于拟合胆固醇时间序列的非线性模式，是用指数模型来拟合这些数据。例如，考虑如下指数趋势方程（18-11） 5  
$$T_t=b_0(b_1)^t$$
Minitab的时间序列模块非常容易用来建立指数趋势方程，不需要涉及对数变化和利用回归分析计算指数趋势方程。图18-15，我们显示了用Minitab的时间序列程序来拟合指数取数方程而得到的计算机输出的图示部分  

In [ ]:
'''
python	statsmodels	Time Series analysis tsa	AutoReg()
python	statsmodels	Time Series analysis tsa	AutoReg.fit()
python	statsmodels	Time Series analysis tsa	AutoRegResults.fittedvalues
python	pandas	series	s.name

'''
chol_armod=AutoReg(chol['Revenue'],1).fit()
chol_estimate=chol_armod.fittedvalues
chol_estimate.name='estimate'
chol_estimate.head()

In [ ]:
'''
python	numpy	Mathematical functions	np.log()
python	numpy	Mathematical functions	np.exp()
'''
chol_formula_exp='np.log(Revenue) ~ Year'
chol_mod_exp=smf.ols(chol_formula_exp,chol).fit()
chol_exp_estimate=np.exp(chol_mod_exp.fittedvalues)
chol_exp_estimate.head()

In [ ]:
'''
5
python	matplotlib	Pyplot function overview	plt.figure() figsize
python	matplotlib	Pyplot function overview	plt.legend() handles
python	matplotlib	Pyplot function overview	plt.legend() labels

'''

plt.figure(figsize=(10,7))
row,=plt.plot([0]+list(chol_estimate.index.values),chol['Revenue'],marker='o')
ar,=plt.plot(chol_estimate.index.values,chol_estimate,marker='o')
lr,=plt.plot(chol_mod.fittedvalues.index.values,chol_mod.fittedvalues,marker='o')
expr,=plt.plot(chol_exp_estimate.index.values,chol_exp_estimate,marker='o')
plt.legend(handles = [row, ar,lr,expr], labels = ['row', 'ar','lr','expr'])
plt.xlim(-1,10)
plt.ylim(0,120)
plt.xticks(np.arange(-1,11))
plt.show()

## 注释
线性趋势回归的基础是找到使预测误差平方和，从而使MSE达到最小的估计的回归方程。所以，就MSE而言，我们期待着线性趋势回归的表现由于Holt线性指数平滑  
当MAE作为预测精度的测度时，线性趋势回归同样提供了更准确的预测;  
然而，基于MAPE，Holt线性指数平滑的表现优于线性趋势回归（实际上并没有） 5  
# 18.5 季节性和趋势
本节我们演示如何对有季节模式的时间序列建立预测。在存在季节性的情形下，我们需要将其并入我们的预测模型以确保预测的准确性  
## 18.5.1 没有趋势的季节性  
作为一个例子，考虑过去5年某服装商店的雨伞销售量。表18-17是时间序列，图18-16是相应的时间序列图  

In [ ]:
umb_lv1.head()

表18-18是雨伞销售量时间序列以及虚拟变量的编码值  

In [ ]:
'''
5
python	sklearn	OneHotEncoder()	OneHotEncoder.fit_transform()
python	scipy	Sparse matrices (scipy.sparse)	csr_matrix.toarray()
'''
ohe=OneHotEncoder()
quart=umb_lv1['Quarter'].values
quart_trans=ohe.fit_transform(quart.reshape(len(quart),1))
quart_df=pd.DataFrame(quart_trans.toarray(),
                      columns=['Q1','Q2','Q3','Q4'])
quart_df.head()

In [ ]:
umb_lv2=umb_lv1.join(quart_df)
umb_lv2.head()

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.summary()
'''
umb_formula_lv2='Sales ~ Q1 + Q2 + Q3 + Q4'
umb_mod_lv2=smf.ols(umb_formula_lv2,umb_lv2).fit()
ubp=umb_mod_lv2.params
umb_mod_lv2.summary()

用表18-18的数据以及Minitab回归程序，我们得到图18-17的计算机输出结果。得到的估计的多元回归方程是  

In [ ]:
'''
5
'''
umb_formula_lv3='Sales ~ Q1 + Q2 + Q3'
umb_mod_lv3=smf.ols(umb_formula_lv3,umb_lv2).fit()
ubp=umb_mod_lv3.params
print('销售量 = {:.1f}{:+.1f}Qrt1{:+.1f}Qrt2{:+.1f}Qrt3'.format(ubp[0],ubp[1],ubp[2],ubp[3]))

有趣的是，我们注意到可以通过计算每个季度雨伞销售量的简单平均数就可以得到下一年的季度预测值，如下表所示  

In [ ]:
umb_lv1.pivot_table(index='Year',columns='Quarter',values='Sales',aggfunc='mean',margins=True)

尽管如此，图18-17的回归输出结果还给出了能平均预测准确性和确定结果显著性的附加信息。同时，对问题境况更复杂的类型，如涉及趋势和季节影响同时存在的时间序列，简单平均方法将无效  
## 18.5.2 季节性和趋势
表18-19是电视机销售量时间序列的数据，图18-18的时间序列图表示在每年第2季度销售量最低，第3季度和第4季度销售量增长  

In [ ]:
'''
5
'''
tv_sale.head()

In [ ]:
tv_sale.plot('year_Q','Sales',figsize=(10,7))
plt.scatter(tv_sale['year_Q'],tv_sale['Sales'])
plt.ylim(0,9)
plt.xlim(0,len(tv_sale['year_Q']))
plt.xticks(np.arange(0,len(tv_sale['year_Q'])+1),tv_sale['year_Q'])
plt.show()

但是时间序列还有为了建立季度销售量的精确预测需要解释的一个上升的线性趋势。这是很容易处理的，只要将季节性的虚拟变量方法与我们讨论的处理线性趋势的时间序列回归方法结合起来即可  
模拟电视机时间序列的季度季节影响和线性趋势的多元回归方程的一般形式如下：  
$$\hat{Y}_t=b_0+b_1Qtr1+b_2Qtr2+b_3Qtr3+b_4t$$
5  
式中,$\hat{Y}_t$为t期销售量的估计值或预测值;Qtr1=1,如果对应于t年的第1季度;否则Qtr1=0；Qtr2=1,如对应于t年的第2季度;否则Qtr2=0,Qtr3=1,如果对应于t的第3季度;否则Qtr3=0;t为时期  
表18-20是电视机销售量时间序列，包含虚拟变量的编码值和时期t。利用表18-20的数据以及Minitab回归程序，我们得到图18-19的计算机输出结果。估计的多元回归方程是（18-12）  

In [ ]:
'''
python	sklearn	OneHotEncoder()	OneHotEncoder()
python	sklearn	OneHotEncoder()	OneHotEncoder.fit_transform()
python	scipy	Sparse matrices (scipy.sparse)	csr_matrix.toarray()
python	pandas	dataframe	d.join()
pandas	pandas	dataframe	d.head()

'''
tv_row_quar=tv_sale['Quarter'].values
tv_tras_quar=ohe.fit_transform(tv_row_quar.reshape(len(tv_row_quar),1))
tv_quar_df=pd.DataFrame(tv_tras_quar.toarray(),
                      columns=['Q1','Q2','Q3','Q4'])
tv_lv1=tv_sale.join(tv_quar_df)
tv_lv1['year_Q']=tv_lv1.index.values+1
tv_lv1.head()

In [ ]:
tv_formula='Sales ~ Q1 + Q2 + Q3 + year_Q'
tv_mod=smf.ols(tv_formula,tv_lv1).fit()
tv_mod.summary()

In [ ]:
'''
5
'''
tv_params=tv_mod.params
print('销售量={:.2f}{:+.2f}Qtr1{:+.2f}Qtr2{:+.3f}Qtr3{:+.3f}t'.format(tv_params[0],
                                                     tv_params[1],
                                                     tv_params[2],
                                                     tv_params[3],
                                                     tv_params[4]))

现在我们用式（18-12）来预测下一年的季度销售量。下一年是电视机销售量时间序列的第5年，即时期为17，18，19，20  

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.predict()
'''
tv_pd_data=tv_mod.predict(pd.DataFrame({'Q1':[1,0,0,0],
                             'Q2':[0,1,0,0],
                             'Q3':[0,0,1,0],
                             'year_Q':[17,18,19,20]}))
tv_pd_data.round(2)

## 18.5.3 基于月度数据的模型
在前面电视机销售量的例子中，我们演示了如何用虚拟变量来说明时间序列的季度季节影响。由于分类变量季度有4个水平，需要3个虚拟变量。然而，许多上午问题用月度而不是用季度进行预测 5  
# 18.6 时间序列分解法
本节我们关注**时间序列分解**的问题。当用这种方法进行预测时，其主要应用是为了获得时间序列一个更好的了解  
了解时间序列的真实状况常常依赖于使用消除季节影响后的数据。许多其他时间序列，如失业率统计、房屋销售量和零售额，都受到很强的季节性影响  
这些数据在做出长期趋势的判断之前，消除季节影响是非常重要的  
时间序列分解方法假设时间序列在t期的实际值$Y_t$，是趋势成分、季节成分和不规则或误差成分这三个成分的函数  
**加法分解模型**的形式如下：（18-13）  
$$Y_t=Trend_t+Seasonal_t+Irregular_t$$
5  
式中，$Trend_t$是t期的趋势值;$Seasonal_t$是t期的季节值;$Irregular_t$是t期的不规则值  
在加法模型中，三个成分的值简单地加在一起得到时间序列的实际值$Y_t$,不规则或误差成分说明时间序列中不能被趋势和季节成分解释的变异性  
加法模型适合于季节影响不依赖于时间序列水平的情形。然而，如果季节影响随时间发生改变，由于长期线性趋势随着销售量的增加逐渐变大，则应该使用乘法模型  
**乘法分解模型**的形式如下（18-14）  
$$Y_t=Trend_t\times Seasonal_t\times Irregular_t$$
5  
式中，$Trend_t$是t期的趋势值;$Seasonal_t$是t期的季节值;$Irregular_t$是t期的不规则值  
在这个模型中，趋势、解决和不规则成分相乘得到时间序列的值  
由于这种方法在实践中常常使用，因此，介绍如何建立趋势和季节成分的估计而对时间序列分解的讨论，我们将只限定于乘法模型  
## 18.6.1 计算季节指数  
确定每个季度的季节影响所使用的计算程序是先计算移动平均数，从数据中剔除组合在一起的季节和不规则影响，留给我们的时间序列只包含趋势和移动平均没有剔除的随机波动  
由于我们使用的数据是季度时间序列，因此在每一次移动平均数的计算中，我们使用4项数据 5  

In [ ]:
'''
python	pandas	Window	Rolling.mean()
'''
tv_roll_sale=tv_sale['Sales'].rolling(4).mean().round(2)
tv_roll_sale.head(10)

当我们回顾5.35的计算时，发现它对应于移动平均数的所有季度的“中间”。但是，请注意，因为是4个季度的移动平均数，不存在中间季度，因此，5.35应该对应于2.5季度，即第2季度的后半部分和第3季度的前半部分  
我们计算出的两个移动平均数并不能直接对应于时间序列的原始季度。因此，我们可以通过计算两个移动平均数的平均数来解决这个难题  
既然第一个移动平均数的中心是2.5季度，第二个移动平均数的中心是3.5解读，则两个移动平均数的平均数的中心为3季度，这应该是它精确的位置。这个移动平均数被称为中心化移动平均数  
表18-21是电视机销售量数据的移动平均数和中心化移动平均数的全面汇总 5  

In [ ]:
'''
python	pandas	series	s.dropna()
'''
tv_cent_sale=tv_roll_sale.rolling(2).mean().dropna()
tv_cent_sale.index=tv_cent_sale.index-2
tv_cent_sale

表18-21的中心化移动平均数能告诉我们时间序列的什么问题呢？图18-20是时间序列实际值和中心化移动平均数的时间序列图  

In [ ]:
tv_sale.plot('year_Q','Sales',figsize=(10,7),marker='o')
tv_cent_sale.plot(marker='o')
plt.ylim(0,9)
plt.xlim(0,len(tv_sale['year_Q']))
plt.xticks(np.arange(0,len(tv_sale['year_Q'])+1),tv_sale['year_Q'])
plt.show()

特别注意到，中心化移动平均数是如何“平滑”实际序列的季节和不规则波动影响的。中心化移动平均数表示数据中的趋势和没有被平滑数据的移动平均剔除掉的随机波动 5  
方程两边同时除以趋势成分$Trend_t$,我们可以识别出时间序列中的季节-不规则的组合影响为  
$$\frac{Y_t}{Trend_t}=Seasonal_t\times Irregular_t$$  
例如，第1年第3季度的趋势值为5.475（中心化移动平均数），因此，6.0/5.475=1.096是季节-不规则的组合值。表18-22汇总了整个时间序列的季节-不规则值  

In [ ]:
'''
python	pandas	series	s.div()
'''
tv_cent_sale.name='center_mean'
tv_irr=tv_sale.join(tv_cent_sale)
tv_irr['irregular']=tv_irr['Sales'].div(tv_irr['center_mean']).round(3)
tv_irr

下面考虑第3季度的季节-不规则值:1.096,1.075和1.109 5  
季节-不规则值大于1.00表明影响在趋势估计之上，而数值小于1.00表明影响在趋势估计之下。因为季节-不规则值每年的波动主要是由于随机误差引起的，所以，我们可以取其平均数以消除不规则影响，从而得到第3季度季节影响的估计值  

In [ ]:
tv_q3_mean=tv_irr[tv_irr['Quarter']==3]['irregular'].dropna().mean()
print('第3季度的影响值是{:.2f}'.format(tv_q3_mean))

我们将1.09称为第3季度的季节指数。在表8-23中，汇总了关于电视机销量时间序列的季节指数的计算结果  

In [ ]:
'''
5
'''
tv_irr_notna=tv_irr.dropna()
tv_qua_imp=tv_irr_notna.groupby('Quarter')['irregular'].mean().round(2)
tv_qua_imp

表18-23中季节指数的解释提供了关于电视机销售量季节成分的一些观测结果。最佳销售季度是第4季度，其销售水平高出趋势估计14%。最差或最少的销售季度是第2季度，其季节指数为0.84，表名其销售水平低于趋势估计16%  
季节成分很明显对应着人们观看电视兴趣的直观期望，由于冬季即将到来人们减少了户外活动，因此电视购买模式趋向于峰值  
在取得季节指数时，对季节指数做最后的调整有时是必要的。乘法模型要求平均季节指数等于1.00，因此，表18-23中的4个季节指数之和必须等于4.00  
## 18.6.2 消除季节影响的时间序列  
剔除了季节影响的时间序列被称为消除季节影响的时间序列，用季节指数剔除时间序列中的季节影响的过程称为消除时间序列的季节影响  
利用乘法分解模型，我们用每一个观测值除以相应的季节指数来消除时间序列的季节影响。乘法分解模型是 5  
$$Y_t=Trend_t\times Seasonal_t\times Irregular_t$$

In [ ]:
tv_qua_imp_s=tv_qua_imp.reset_index()
tv_qua_imp_s

In [ ]:
'''
python	pandas	dataframe	d.merge() left_index
'''
tv_sale_eli=tv_sale.merge(tv_qua_imp_s,how='left',on='Quarter',left_index=True)
tv_sale_eli['adj_sales']=tv_sale_eli['Sales'].div(tv_sale_eli['irregular']).round(2)
tv_sale_eli.head(10)

因此，当时间序列的每个观察值$Y_t$除以相应的季节指数时，剩下的数据中只包含趋势和随机波动（不规则成分），图18-21是消除季节影响的时间序列图    

In [ ]:
'''
5
'''
tv_sale_eli.plot('year_Q','adj_sales',marker='o')
plt.ylim(0,9)
plt.xlim(0,len(tv_sale['year_Q']))
plt.xticks(np.arange(0,len(tv_sale_eli['year_Q'])+1),tv_sale_eli['year_Q'])
plt.show()

因此，当时间序列的每个观察值$Y_t$除以相应的季节指数时，剩下的数据中只包含趋势和随机波动（不规则成分）  
## 18.6.3 利用消除季节影响的时间序列确定趋势  
出现在图18-21中的消除季节影响的电视机销售量时间序列图，显示有一个向上的线性趋势。为了确定这个趋势，我们将使用18.4节介绍的同样的方法，用一个线下趋势方程来拟合消除季节影响的时间序列  
图18-22是利用Minitab回归分析方程来估计消除季节影响的电视机销售量时间序列趋势性的计算机输出结果。估计的线性趋势方程为  

In [ ]:
'''
python	pandas	dataframe	d.reset_index() drop

'''
tv_sale_1863a=tv_sale_eli.reset_index(drop=True)
tv_sale_1863a['year_rank']=tv_sale_1863a.index.values+1
tv_sale_1863a.head()

In [ ]:
'''
5
'''
tv_formula_1863a='adj_sales ~ year_rank'
tv_mod_1863a=smf.ols(tv_formula_1863a,tv_sale_1863a).fit()
tv_mod_1863a.summary()